<a href="https://colab.research.google.com/github/ferasraed/Scraping-Over-Google-Scholar/blob/main/CIS711_Case_Stady.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests as rs
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

In [7]:
scholars = []
header = ['Name', 'University/Company', 'Position', 'Homepage', 'Research Interests', 'Citations', 'h-index', 'i10-index', 'First Publication', 'Co_authors for First Publication', 'Citation for First Publication', 'Second Publication', 'Co_authors for Second Publication', 'Citation for Second Publication']
scholars.append(header)

def collect_data(soup):
    try:
        div_name = soup.find('div', id='gsc_prf_in')
        name = div_name.text.strip()
    except AttributeError:
        name = ''

    try:
        div_company = soup.find('a', class_='gsc_prf_ila')
        if div_company:
            company = div_company.text.strip()
            if company.lower() == 'homepage':
                company = ''
        else:
            div_company = soup.find('div', class_='gsc_prf_il')
            company_text = div_company.get_text().strip()
            company = company_text.split(',')[0]
            if company.lower() == 'homepage':
                company = ''
    except AttributeError:
        company = ''

    try:
        div_position = soup.find('div', class_='gsc_prf_il')
        position = div_position.get_text(strip=True).split(',')[0]
    except AttributeError:
        position = ''

    try:
        a_homepage = soup.find('div', id='gsc_prf_ivh').find('a', class_='gsc_prf_ila')
        homepage_url = a_homepage.get('href')
    except AttributeError:
        homepage_url = ''

    try:
        td_citations = soup.find('td', class_='gsc_rsb_std')
        citations = td_citations.text.strip()
    except AttributeError:
        citations = ''

    try:
        td_h_index = soup.find('td', string='h-index')
        h_index_value = td_h_index.find_next_sibling('td').text.strip()
    except AttributeError:
        h_index_value = ''

    try:
        td_i10_index = soup.find('td', string='i10-index')
        i10_index_value = td_i10_index.find_next_sibling('td').text.strip()
    except AttributeError:
        i10_index_value = ''

    try:
        research_interests_div = soup.find('div', id='gsc_prf_int')
        research_interests = [interest.text.strip() for interest in research_interests_div.find_all('a', class_='gsc_prf_inta')]
    except AttributeError:
        research_interests = []
    title1 = ''
    co_authors1 = ''
    citation1 = ''
    title2 = ''
    co_authors2 = ''
    citation2 = ''
    try:
        publications = soup.find_all('tr', class_='gsc_a_tr')[:1]
        for publication in publications:
            title1 = publication.find('a', class_='gsc_a_at').get_text()
            co_authors1 = publication.find('div', class_='gs_gray').get_text()
            citation1 = publication.find('a', class_='gsc_a_ac').get_text()
    except AttributeError:
        pass

    try:
        publications = soup.find_all('tr', class_='gsc_a_tr')[:2]
        for publication in publications:
            title2 = publication.find('a', class_='gsc_a_at').get_text()
            co_authors2 = publication.find('div', class_='gs_gray').get_text()
            citation2 = publication.find('a', class_='gsc_a_ac').get_text()
    except AttributeError:
        pass

    row = [name, company, position, homepage_url, research_interests, citations, h_index_value, i10_index_value,
          title1, co_authors1, citation1, title2, co_authors2, citation2]
    scholars.append(row)

In [ ]:
URLs = ['https://scholar.google.com/citations?hl=en&user=kukA0LcAAAAJ', 'https://scholar.google.com/citations?hl=en&user=ALVSZAYAAAAJ', 'https://scholar.google.com/citations?hl=en&user=Tb0ZrYwAAAAJ', 'https://scholar.google.com/citations?hl=en&user=LnBGQ3QAAAAJ', 'https://scholar.google.com/citations?hl=en&user=FaOcyfMAAAAJ', 'https://scholar.google.com/citations?hl=en&user=OQpf9YsAAAAJ', 'https://scholar.google.com/citations?hl=en&user=hBTsXMgAAAAJ', 'https://scholar.google.com/citations?hl=en&user=wsGvgA8AAAAJ', 'https://scholar.google.com/citations?hl=en&user=wsGvgA8AAAAJ', 'https://scholar.google.com/citations?hl=en&user=tvUH3WMAAAAJ']

visited_URL = 0
visited_URLs = set()
def extract_coauthor_urls(profile_url):
    global visited_URL
    header_k = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
    if profile_url not in visited_URLs:
        resquested = rs.get(profile_url, headers=header_k)
        visited_URL += 1
        html_content = resquested.content
        soup = BeautifulSoup(html_content, 'html.parser')
        for co_author in soup.find_all('div', class_='gsc_rsb_aa'):
            co_author_url = co_author.find('a').get('href')
            co_author_url = urllib.parse.urljoin('https://scholar.google.com/', co_author_url)
            if co_author_url in URLs:
              continue
            URLs.append(co_author_url)

        # get the data
        collect_data(soup)

for URL in URLs:
    if visited_URL >= 60000:
        break

    extract_coauthor_urls(URL)

In [8]:
header = scholars[0]
scholars = scholars[1:]
scholars_df = pd.DataFrame(scholars, columns=header)
scholars_df = scholars_df.drop_duplicates(subset=['Research Interests'])
scholars_df.to_csv("Scholars_info.csv", index=False)